In [ ]:
from ucimlrepo import fetch_ucirepo 

from sklearn.model_selection import train_test_split
from normalizer import Normalizer
import pandas as pd
import numpy as np



In [ ]:
mushroom = fetch_ucirepo(id=848) 
  
attributes_type = mushroom.variables.type
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 

In [26]:
import importlib
import decision_tree
importlib.reload(decision_tree)  # Ricarica il modulo


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

normalizer = Normalizer(n_bins=4, normalization="minmax")

X_train = normalizer.normalize(X_train)
y_train = normalizer._fix_label(y_train)


#y_train = y_train.to_numpy().flatten()

# Decision tree classifier
dt = decision_tree.Decision_tree(splitting_criteria = 'entropy', max_depth = 18, max_leaves = 5, min_samples_split=2)

dt.fit(X_train, y_train)

sasas
cap-diameter True
cap-shape False
cap-surface False
cap-color False
does-bruise-or-bleed False
gill-attachment False
gill-spacing False
gill-color False
stem-height True
stem-width True
stem-root False
stem-surface False
stem-color False
veil-type False
veil-color False
has-ring False
ring-type False
spore-print-color False
habitat False
season False


AttributeError: 'Decision_tree' object has no attribute '_grow_tree'